In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## IMPORT LIBRARIES AND DATASETS

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from sklearn import datasets

data = datasets.load_iris()
dir(data)

['DESCR',
 'data',
 'data_module',
 'feature_names',
 'filename',
 'frame',
 'target',
 'target_names']

In [4]:
df = pd.DataFrame(data.data, columns = data.feature_names)
df['target'] = data.target

df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [5]:
X = df.drop(columns = ['target'])
y = df['target']

X.shape, y.shape

((150, 4), (150,))

## DATAPROCESSING

In [6]:
from sklearn.preprocessing import MinMaxScaler

s = MinMaxScaler()
X[:] = s.fit_transform(X)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

X_train.shape, y_train.shape

((120, 4), (120,))

In [8]:
from sklearn.svm import SVC

model1 = SVC(kernel = 'rbf', C = 30, gamma = 'auto')
model1.fit(X_train, y_train)
model1.score(X_test, y_test)

0.9666666666666667

## HYPER PARAMETER TUNING - K-FOLD VALIDATION

In [9]:
from sklearn.model_selection import cross_val_score

cross_val_score(SVC(kernel = 'linear', C = 10, gamma= 'auto'), X, y, cv = 10)

array([1.        , 1.        , 1.        , 1.        , 0.86666667,
       0.93333333, 0.86666667, 1.        , 1.        , 1.        ])

In [10]:
cross_val_score(SVC(kernel = 'rbf', C = 10, gamma= 'auto'), X, y, cv = 10)

array([1.        , 1.        , 1.        , 1.        , 0.93333333,
       0.93333333, 0.86666667, 1.        , 1.        , 1.        ])

In [17]:
kernels = ['rbf', 'linear']
C = [1,10,20]
avg_scores = {}
for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(SVC(kernel=kval,C=cval,gamma='auto'),X, y, cv=10)
        avg_scores[kval + '_' + str(cval)] = np.average(cv_scores)

avg_scores

{'rbf_1': 0.9666666666666666,
 'rbf_10': 0.9733333333333334,
 'rbf_20': 0.9666666666666668,
 'linear_1': 0.9666666666666666,
 'linear_10': 0.9666666666666668,
 'linear_20': 0.9600000000000002}

## GRID SEARCH TO FIND BEST MODEL

In [19]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(SVC(gamma = 'auto'), {
    'C': [1, 10, 20],
    'kernel': ['rbf','linear']
}, cv = 10, return_train_score = False)

clf.fit(X, y)
clf.cv_results_

{'mean_fit_time': array([0.00300407, 0.00335827, 0.00273352, 0.00219107, 0.00219998,
        0.00217333]),
 'std_fit_time': array([0.0004442 , 0.00051647, 0.00035234, 0.00013128, 0.00017865,
        0.00010382]),
 'mean_score_time': array([0.00204957, 0.00212476, 0.00202708, 0.00152113, 0.00152562,
        0.00153828]),
 'std_score_time': array([3.49510549e-04, 2.86413668e-04, 5.69031089e-04, 4.95948985e-05,
        9.11270274e-05, 1.06721813e-04]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20

In [21]:
df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003004,0.000444,0.002050,0.000350,1,rbf,"{'C': 1, 'kernel': 'rbf'}",1.0,0.933333,1.0,0.933333,1.000000,0.933333,0.866667,1.0,1.0,1.0,0.966667,0.044721,4
1,0.003358,0.000516,0.002125,0.000286,1,linear,"{'C': 1, 'kernel': 'linear'}",1.0,0.933333,1.0,0.933333,1.000000,0.933333,0.866667,1.0,1.0,1.0,0.966667,0.044721,4
2,0.002734,0.000352,0.002027,0.000569,10,rbf,"{'C': 10, 'kernel': 'rbf'}",1.0,1.000000,1.0,1.000000,0.933333,0.933333,0.866667,1.0,1.0,1.0,0.973333,0.044222,1
3,0.002191,0.000131,0.001521,0.000050,10,linear,"{'C': 10, 'kernel': 'linear'}",1.0,1.000000,1.0,1.000000,0.866667,0.933333,0.866667,1.0,1.0,1.0,0.966667,0.053748,2
4,0.002200,0.000179,0.001526,0.000091,20,rbf,"{'C': 20, 'kernel': 'rbf'}",1.0,0.933333,1.0,1.000000,0.866667,1.000000,0.866667,1.0,1.0,1.0,0.966667,0.053748,2
5,0.002173,0.000104,0.001538,0.000107,20,linear,"{'C': 20, 'kernel': 'linear'}",1.0,1.000000,1.0,0.933333,0.866667,0.933333,0.866667,1.0,1.0,1.0,0.960000,0.053333,6


In [23]:
df[['param_C','param_kernel','mean_test_score', 'rank_test_score']]

,param_C,param_kernel,mean_test_score,rank_test_score
0,1,rbf,0.966667,4
1,1,linear,0.966667,4
2,10,rbf,0.973333,1
3,10,linear,0.966667,2
4,20,rbf,0.966667,2
5,20,linear,0.960000,6


In [24]:
dir(clf)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_feature_names',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_estimator_type',
 '_format_results',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run_search',
 '_select_best_index',
 '_validate_data',
 '_validate_params',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 'classes_',
 'cv',
 'cv_results_',
 'decision_function',
 'error_score',
 'estimator',
 'feature_names_in_',
 'fit',
 'get_params',
 'inverse_transform',
 

In [25]:
clf.best_score_

0.9733333333333334

In [26]:
clf.best_params_

{'C': 10, 'kernel': 'rbf'}

In [30]:
# The gridsearch takes all the combination, which directly increases the computation power of the machine, and it drains energy, so we use randomised search strategy

from sklearn.model_selection import RandomizedSearchCV

rs = RandomizedSearchCV(SVC(gamma='auto'), {
        'C': [1,10,20],
        'kernel': ['rbf','linear']
    }, 
    cv=5, 
    return_train_score=False, 
    n_iter=5
)

rs.fit(X, y)
pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score', 'rank_test_score']]

,param_C,param_kernel,mean_test_score,rank_test_score
0,1,rbf,0.960000,4
1,20,rbf,0.966667,1
2,10,linear,0.966667,1
3,1,linear,0.966667,3
4,20,linear,0.960000,4


In [34]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    }
}

In [36]:
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(X, y)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.966667,"{'C': 10, 'kernel': 'rbf'}"
1,random_forest,0.966667,{'n_estimators': 5}
2,logistic_regression,0.920000,{'C': 10}
